# LangChain 0.1.0 Quickstart - Create Your Own RAG Chains

Welcome to this Quickstart walkthrough of the new version of Langchain. This tutorial follows the official Langchain documentation quickstart. I made a video where I go through exactly all the code in this notebook 👉 https://youtu.be/LBNpyjcbv0o

# 1.0 Install the required packages

Some packages have changed inside Langchain. This is how you should install and import them from now on. Remember that langchain has been divided into three main packages:
- `langchain` contains pre-built chains and other stuff you may need.
- `langchain-core` contains the main abstractions of the library.
- `langchain-community` contains third-party integrations like Chroma, FAISS, etc.
- Some third-party integrations have their own package (outside `langchain-community`) that you should install if you want to use them. For example, OpenAI has its own package: `langchain-openai`.

All three are installed when you run `pip install langchain`. But you can install community or core individually as well.

In [1]:
# ! pip install langchain

In [3]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import os

from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    model_name="gemini-pro", # Replace with your desired Gemini model
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)


def generate_pro(input_prompt):
    model = GenerativeModel("gemini-pro")
    full_prompt = '''summarize the prompt below and do note prompt below will be send to imagen mode so please clean up any sensitve words and replace them into unblocked words like replace girl or lady can be replaced by female human and so on : ''' + input_prompt
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 8190,
        "temperature": 0.2,
        "top_p": 1
    },stream=False,)
    
    # print (responses.text)
    
    return(responses.text)

In [5]:
llm.invoke("what is docker and how is it useful for deployment?")

AIMessage(content="**What is Docker?**\n\nDocker is an open-source platform that uses containerization technology to package, distribute, and run applications in isolated environments called containers. A container bundles the application code, its dependencies, and its runtime environment into a single executable unit.\n\n**How Docker is Useful for Deployment:**\n\nDocker simplifies deployment in several ways:\n\n**1. Isolation and Consistency:**\n* Containers provide isolated environments, ensuring that applications run without interfering with each other or the host system.\n* This consistency ensures that applications behave the same in different environments, from development to production.\n\n**2. Rapid Deployment:**\n* Containers are lightweight and portable, enabling fast and efficient deployment.\n* They can be easily copied, moved, and launched across different servers or cloud platforms.\n\n**3. Scalability and High Availability:**\n* Docker allows easy scaling of applicatio

# 2.0 Create your first chain with LCEL

LCEL is now the default way to create chains in LangChain. It has a more pipeline-like syntax and allows you to modify already-existing chains.

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt_input = "You are an English-French translator that return whatever the user says in French" 
prompt = ChatPromptTemplate.from_messages([
    ("user", prompt_input)
])

In [11]:
chain = prompt | llm

In [12]:
chain.invoke({
    "input": "i enjoy going to rock concerts"
    })

AIMessage(content="Tu es un traducteur anglais-français qui renvoie tout ce que l'utilisateur dit en français", response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'citation_metadata': None, 'usage_metadata': {'prompt_token_count': 15, 'candidates_token_count': 21, 'total_token_count': 36}})

In [13]:
# add output parser to the chain

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [14]:
chain = prompt | llm | output_parser

In [15]:
chain.invoke({"input": "my friend robert has a blue cat"})

"Vous êtes un traducteur anglais-français qui renvoie tout ce que l'utilisateur dit en français."

In [16]:
llm.invoke("what is new in langchain 0.1.0?")

AIMessage(content='**New Features**\n\n* **Model support:**\n    * Added support for multiple models.\n    * **CLIP**: A powerful image-text model from OpenAI.\n    * **GPT-J**: A large language model from EleutherAI.\n    * **T5**: A text-to-text transfer transformer model from Google AI.\n    * **BART**: A sequence-to-sequence model from Facebook AI Research.\n    * **mBART**: A multilingual sequence-to-sequence model from Facebook AI Research.\n* **Chain support:**\n    * **Chainable models**: Models can now be chained together to create complex language processing pipelines.\n    * **Dynamic chain assembly**: Chains can be assembled dynamically at runtime.\n* **Input and output handling:**\n    * **Unified input representation**: Inputs are now represented as a single `InputSequence` object.\n    * **Unified output representation**: Outputs are now represented as a single `OutputSequence` object.\n    * **Streamlined input/output conversion**: Easy conversion between different inpu

# 3.0 Create a Retrieval Chain


## 3.1 Load the source documents

First, we will have to load the documents that will enrich our LLM prompt. We will use [this blog post](https://blog.langchain.dev/langchain-v0-1-0/) from LangChain's official website explaining the new release. OpenAI's models were not trained on this content, so the only way to ask questions about it is to build a RAG chain.

The first thing to do is to load the blog content to our vector store. We will use beautiful soup to scrap the blog post. Then we will store it in a FAISS vector store.

In [17]:
# retrieval chain

% pip install beautifulsoup4

UsageError: Line magic function `%` not found.


In [ ]:
# ! pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 53.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/langchain-v0-1-0/")

docs = loader.load()

In [ ]:
docs

[Document(page_content='\n\n\nLangChain v0.1.0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n                LangChain Blog\n        \n\n\n\n\n\n\nHome\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\nCase Studies\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangChain v0.1.0\n\n10 min read\nJan 8, 2024\n\n\n\n\n\nToday we‚Äôre excited to announce the release of langchain 0.1.0, our first stable version. It is fully backwards compatible, comes in both Python and JavaScript, and comes with improved focus through both functionality and documentation. A stable version of LangChain helps us earn developer trust and gives us the ability to evolve the library systematically and safely.Python GitHub DiscussionPython v0.1.0 GuidesJS v0.1.0 GuidesYouTube WalkthroughIntroductionLangChain has been around for a little over a year and has changed a lot as it‚Äô

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter  import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)


In [ ]:
documents

[Document(page_content='LangChain v0.1.0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n                LangChain Blog\n        \n\n\n\n\n\n\nHome\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\nCase Studies\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangChain v0.1.0\n\n10 min read\nJan 8, 2024', metadata={'source': 'https://blog.langchain.dev/langchain-v0-1-0/', 'title': 'LangChain v0.1.0', 'language': 'en'}),
 Document(page_content='Today we‚Äôre excited to announce the release of langchain 0.1.0, our first stable version. It is fully backwards compatible, comes in both Python and JavaScript, and comes with improved focus through both functionality and documentation. A stable version of LangChain helps us earn developer trust and gives us the ability to evolve the library systematically and safely.Python GitHub DiscussionPython v0.1.0 GuidesJS

In [ ]:
vectorstore = FAISS.from_documents(documents, embeddings)

## 3.2 Create a Context-Aware LLM Chain

Here we create a chain that will answer a question given a context. For now, we are passing the context manually, but in the next step we will pass in the documents fetched from the vector store we created above 👆

In [ ]:
# create chain for documents

from langchain.chains.combine_documents import create_stuff_documents_chain

template = """"Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "what is langchain 0.1.0?",
    "context": [Document(page_content="langchain 0.1.0 is the new version of a llm app development framework.")]
})

'Langchain 0.1.0 is the new version of a llm app development framework.'

## 3.2 Create the RAG Chain

RAG stands for Retrieval-Augmented Generation. This means that we will enrich the prompt that we send to the LLM. We will use with the documents that wil will retrieve from the vector store for this. LangChain comes with the function `create_retrieval_chain` that allows you to create one of these.

In [ ]:
# create retrieval chain

from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({
    "input": "what is new in langchain 0.1.0"
})

In [ ]:
response['answer']

'In langchain 0.1.0, the new features include improved focus through both functionality and documentation, a new versioning standard, the separation of langchain-core and partner packages, and the introduction of LangSmith for debugging LLM applications.'

# 4.0 Create Conversational RAG Chain

Now we will create exactly the same thing as above, but we will have the AI assistant take the history of the conversation into account. In short, we will build the same chain as above but with we will take into account the previous messages in these two steps of the chain:

- When fetching the documents from the vector store. We will fetch documents related to the entire conversation and not just the latest message.
- When answering the question. We will send to the LLM the history of the conversation along the context and query.





## 4.1 Create a Conversation-Aware Retrieval Chain

This chain will return the documents related to the entire conversation and not just the latest message.

In [ ]:
# conversational retrieval chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Is there anything new about Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it!"
})

[Document(page_content='Today we‚Äôre excited to announce the release of langchain 0.1.0, our first stable version. It is fully backwards compatible, comes in both Python and JavaScript, and comes with improved focus through both functionality and documentation. A stable version of LangChain helps us earn developer trust and gives us the ability to evolve the library systematically and safely.Python GitHub DiscussionPython v0.1.0 GuidesJS v0.1.0 GuidesYouTube WalkthroughIntroductionLangChain has been around for a little over a year and has changed a lot as it‚Äôs grown to become the default framework for building LLM applications. As we previewed a month ago, we recently decided to make significant changes to the\xa0 LangChain package architecture in order to better organize the project and strengthen the foundation.\xa0Specifically we made two large architectural changes: separating out langchain-core and separating out partner packages (either into langchain-community or standalone p

## 4.2 Use Retrieval Chain together with Document Chain

Now we will create a document chain that contains a placeholder for the conversation history. This placeholder will be populated with the conversation history that we will pass as its value. We will the plug it together with the retriever chain we created above to have a conversational retrieval-augmented chain.

In [ ]:
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversational_retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
response = conversational_retrieval_chain.invoke({
    'chat_history': [],
    "input": "What is langchain 0.1.0 about?"
})

In [ ]:
response

{'chat_history': [],
 'input': 'What is langchain 0.1.0 about?',
 'context': [Document(page_content='Today we‚Äôre excited to announce the release of langchain 0.1.0, our first stable version. It is fully backwards compatible, comes in both Python and JavaScript, and comes with improved focus through both functionality and documentation. A stable version of LangChain helps us earn developer trust and gives us the ability to evolve the library systematically and safely.Python GitHub DiscussionPython v0.1.0 GuidesJS v0.1.0 GuidesYouTube WalkthroughIntroductionLangChain has been around for a little over a year and has changed a lot as it‚Äôs grown to become the default framework for building LLM applications. As we previewed a month ago, we recently decided to make significant changes to the\xa0 LangChain package architecture in order to better organize the project and strengthen the foundation.\xa0Specifically we made two large architectural changes: separating out langchain-core and sep

In [ ]:
response['answer']

'LangChain 0.1.0 is the first stable version of the LangChain framework. It is fully backwards compatible and available in both Python and JavaScript. This release focuses on improving functionality and documentation. The stable version of LangChain aims to earn developer trust and provide a foundation for systematic and safe evolution of the library.'

In [ ]:
# simulate conversation history

chat_history = [
    HumanMessage(content="Is there anything new about Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

response = conversational_retrieval_chain.invoke({
    'chat_history': chat_history,
    "input": "Tell me more about it!"
})

In [ ]:
response

{'chat_history': [HumanMessage(content='Is there anything new about Langchain 0.1.0?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me more about it!',
 'context': [Document(page_content='Today we‚Äôre excited to announce the release of langchain 0.1.0, our first stable version. It is fully backwards compatible, comes in both Python and JavaScript, and comes with improved focus through both functionality and documentation. A stable version of LangChain helps us earn developer trust and gives us the ability to evolve the library systematically and safely.Python GitHub DiscussionPython v0.1.0 GuidesJS v0.1.0 GuidesYouTube WalkthroughIntroductionLangChain has been around for a little over a year and has changed a lot as it‚Äôs grown to become the default framework for building LLM applications. As we previewed a month ago, we recently decided to make significant changes to the\xa0 LangChain package architecture in order to better organize the project and strengthen the foundation.\xa0Sp

In [ ]:
response['answer']

"LangChain 0.1.0 is the first stable version of the LangChain framework. It is fully backwards compatible and is available in both Python and JavaScript. This release comes with improved functionality and documentation, providing a more focused and reliable experience for developers. \n\nOne significant change in LangChain 0.1.0 is the separation of langchain-core and partner packages. Langchain-core contains the main abstractions, interfaces, and core functionality, while partner packages are either included in langchain-community or exist as standalone partner packages. This architectural change helps organize the project and strengthen its foundation. \n\nTo address the challenges posed by previous minor version releases (0.0.x), LangChain now follows a new versioning standard. Any breaking changes to the public API will result in a minor version bump (second digit), while bug fixes and new features will result in a patch version bump (third digit). This versioning standard aims to 

# Conclusion

Congratulations for finishing this tutorial. For a more in-depth walk-through, you can go to the official [LangChain documentation](https://python.langchain.com/docs/get_started/quickstart).

In the meantime, be sure to subscribe to [my YouTube channel](https://www.youtube.com/@alejandro_ao). I upload free tutorials and courses to help you become an AI Engineer.